In [70]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import f1_score
from mpl_toolkits.mplot3d import Axes3D
from plotly.offline import init_notebook_mode, iplot


In [69]:
train =  pd.read_csv('/Users/erica.stevenson/ML_projects/Predict_Student_Performance/predict-student-performance-from-game-play/train.csv')
labels =  pd.read_csv('/Users/erica.stevenson/ML_projects/Predict_Student_Performance/predict-student-performance-from-game-play/train_labels.csv')
test =  pd.read_csv('/Users/erica.stevenson/ML_projects/Predict_Student_Performance/predict-student-performance-from-game-play/test.csv')
sample_submission = pd.read_csv("/Users/erica.stevenson/ML_projects/Predict_Student_Performance/predict-student-performance-from-game-play/sample_submission.csv")

In [72]:
print(train.shape)
train.sample(10)

(13174211, 20)


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
3504482,21010410191823096,329,611492,person_click,basic,8,NaN,-129.334393,65.000000,262.0,265.0,NaN,Your gramps is awesome! Always full of stories.,worker,tunic.drycleaner.frontdesk,tunic.drycleaner.frontdesk.worker.hub,NaN,NaN,NaN,5-12
6183566,21040108533017056,16,14520,navigate_click,undefined,0,NaN,-885.405288,-248.557882,22.0,525.0,NaN,NaN,notebook,tunic.historicalsociety.closet,NaN,NaN,NaN,NaN,0-4
1074302,20110219371354490,63,84584,navigate_click,undefined,2,NaN,-631.508852,-185.703866,132.0,426.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
13125867,22090611560551024,155,357458,notebook_click,open,4,1.0,-559.144268,-294.555556,52.0,1022.0,NaN,NaN,NaN,tunic.kohlcenter.halloffame,NaN,NaN,NaN,NaN,0-4
9946715,21110013234220824,594,1482414,cutscene_click,basic,16,NaN,-1168.489245,-148.695451,221.0,461.0,NaN,I told you!,confrontation,tunic.historicalsociety.cage,tunic.historicalsociety.cage.confrontation,NaN,NaN,NaN,13-22
6824449,21040318540158252,1312,1642326,person_click,basic,21,NaN,350.831814,251.557377,842.0,87.0,NaN,Wait a minute...,archivist_glasses,tunic.historicalsociety.frontdesk,tunic.historicalsociety.frontdesk.archivist_gl...,NaN,NaN,NaN,13-22
9670036,21100308564536628,177,594901,navigate_click,undefined,5,NaN,614.003093,320.137215,536.0,131.0,NaN,NaN,tobasement,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,5-12
6431292,21040209394329296,917,1109692,person_click,basic,19,NaN,107.408735,103.519488,491.0,224.0,NaN,I'm investigating this flag.,flag_girl,tunic.flaghouse.entry,tunic.flaghouse.entry.flag_girl.hello,NaN,NaN,NaN,13-22
5252278,21030114465206956,93,91769,navigate_click,undefined,2,NaN,-410.778272,-89.315587,370.0,549.0,NaN,NaN,NaN,tunic.historicalsociety.entry,NaN,NaN,NaN,NaN,0-4
5323163,21030209301382504,633,1370124,person_click,basic,18,NaN,432.630371,-441.046096,474.0,361.0,NaN,"Oh yeah, cranes eat insects!",crane_ranger,tunic.wildlife.center,tunic.wildlife.center.crane_ranger.crane,NaN,NaN,NaN,13-22


In [73]:
# how many unique session ids, rooms, texts?
print(f"number of users = {len(train.session_id.unique())}, number of rooms = {len(train.room_fqid.unique())}, number of texts = {len(train.text_fqid.unique())}")

number of users = 11779, number of rooms = 19, number of texts = 127


In [89]:
print(labels.shape)
labels.sample(10)

(212022, 8)


,session_id,correct,q_num,m,p1,r,p2,p3
72171,110611361645720,0,7,0.729264,1.0,0.580248,1,NaN
49872,1010210233619852,1,5,0.546396,1.0,0.778860,0,NaN
210038,2010315520993850,1,18,0.950590,1.0,0.596133,1,NaN
183498,1050413542173336,1,16,0.737839,1.0,0.000851,1,NaN
13771,1000313555418900,1,2,0.978776,1.0,0.174553,1,NaN
105605,2080313540030892,1,9,0.735461,1.0,0.305673,1,NaN
174005,1110416360750336,0,15,0.482978,0.0,0.048983,1,NaN
163173,2020408082026628,1,14,0.710077,1.0,0.321030,1,NaN
136485,1050511164586204,1,12,0.857458,1.0,0.449495,1,NaN
8296,1090418223662690,1,1,0.724000,1.0,0.340279,1,1.0


### cleanup of datasets

In [75]:
# LABELS
# split session id column into session number and question number
labels['q_num'] = labels['session_id'].apply(lambda x: int(x.split('_')[-1][1:]))
labels['session_id'] = labels['session_id'].apply(lambda x: int(x.split('_')[0][1:]))
labels.sample(10)

,session_id,correct,q_num
23890,100314474416090,1,3
139208,2000614344160724,1,12
205195,1030311420587996,1,18
90985,1100120335973210,0,8
189599,110315075280890,1,17
53824,1050312261464030,1,5
159983,1050509104593910,1,14
27147,1020111512658210,1,3
23741,100209120881110,1,3
172788,1080511260408920,1,15


In [76]:
# TRAIN / TEST
# drop columns with only NaN values
train.drop(['fullscreen','hq','music'],axis=1)
test.drop(['fullscreen','hq','music'],axis=1)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13174206,22100221145014656,1600,5483231,navigate_click,undefined,22,NaN,343.887291,36.701026,483.0,273.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,13-22
13174207,22100221145014656,1601,5485166,navigate_click,undefined,22,NaN,332.696070,141.493178,545.0,221.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22
13174208,22100221145014656,1602,5485917,navigate_click,undefined,22,NaN,369.912859,140.569205,611.0,217.0,NaN,NaN,NaN,tunic.capitol_2.hall,NaN,13-22
13174209,22100221145014656,1603,5486753,navigate_click,undefined,22,NaN,252.299653,123.805889,526.0,232.0,NaN,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,13-22


### first exploration of the data

In [90]:
# what's the count for each event name?
event_count = train.event_name.value_counts().reset_index()
event_count.columns = ['event_name', 'count']

fig = px.bar(event_count, x='event_name', y='count', color='count', title="Count the number of occurrences of each event")
fig.show()

In [78]:
name_count = train['name'].value_counts().reset_index()
name_count.columns = ['name', 'count']

fig = px.bar(name_count, x='name', y='count', color='count', title="Count the number of occurrences of each name")
fig.show()

In [79]:
page_count = train['page'].value_counts().reset_index()
page_count.columns = ['page', 'count']

fig = px.bar(page_count, x='page', y='count', color='count', title="Count the number of occurrences of each page")
fig.show()

In [80]:
train[train.level==20].head(20)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
800,20090312431273200,851,1155378,navigate_click,undefined,20,NaN,257.404971,20.519535,641.0,307.0,NaN,NaN,flag_girl,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
801,20090312431273200,852,1156443,person_click,basic,20,NaN,407.406325,-106.480477,791.0,434.0,NaN,"If I were you, I'd go to the library and do so...",flag_girl,tunic.flaghouse.entry,tunic.flaghouse.entry.flag_girl.symbol_recap,NaN,NaN,NaN,13-22
802,20090312431273200,853,1156877,navigate_click,undefined,20,NaN,407.406448,-106.480478,791.0,434.0,NaN,NaN,NaN,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
803,20090312431273200,854,1157727,navigate_click,undefined,20,NaN,486.140191,-62.467998,815.0,387.0,NaN,NaN,tomap,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
804,20090312431273200,855,1159027,map_hover,basic,20,NaN,NaN,NaN,NaN,NaN,100.0,NaN,tunic.drycleaner,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
805,20090312431273200,856,1159226,map_hover,basic,20,NaN,NaN,NaN,NaN,NaN,166.0,NaN,tunic.capitol_2,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
806,20090312431273200,857,1159661,map_click,undefined,20,NaN,58.957046,24.393992,325.0,299.0,NaN,NaN,tunic.library,tunic.flaghouse.entry,NaN,NaN,NaN,NaN,13-22
807,20090312431273200,858,1160893,navigate_click,undefined,20,NaN,-393.219992,122.090406,437.0,236.0,NaN,NaN,worker,tunic.library.frontdesk,NaN,NaN,NaN,NaN,13-22
808,20090312431273200,859,1199469,person_click,basic,20,NaN,-165.356629,89.863411,421.0,261.0,NaN,"Welcome back, Dear! How can I help you?",worker,tunic.library.frontdesk,tunic.library.frontdesk.worker.flag,NaN,NaN,NaN,13-22
809,20090312431273200,860,1200256,person_click,basic,20,NaN,-169.629770,85.954557,417.0,264.0,NaN,I need to learn more about this flag!,worker,tunic.library.frontdesk,tunic.library.frontdesk.worker.flag,NaN,NaN,NaN,13-22


## Predict Train Means Baseline

In Kaggle's Predict Student Performance Competition, we are asked to predict whether a student gets each of 18 questions correct or incorrect. In this notebook we compute the mean of correctness for each of the 18 questions from train data. Then we use this mean to predict 1s and 0s for each of the test users per question.

### Compute Train Means

In [81]:
# create a dictionary to use for mapping question number to mean of correct answers 
question_means = labels.groupby('q_num').correct.agg('mean').to_dict()
question_means

{1: 0.7240003395874013,
 2: 0.9787757874182867,
 3: 0.9321674165888445,
 4: 0.7993038458273198,
 5: 0.5463961287036251,
 6: 0.7720519568724,
 7: 0.7292639443076662,
 8: 0.6143136089651074,
 9: 0.7354614143815265,
 10: 0.5003820358264708,
 11: 0.6441973002801596,
 12: 0.857458188301214,
 13: 0.27048136514135324,
 14: 0.7100772561337975,
 15: 0.482978181509466,
 16: 0.7378385261906784,
 17: 0.6852873758383564,
 18: 0.9505900331097716}

### Compute Validation Score

The competition metric is f1_score macro. We need to convert probability predictions into predictions of 1s and 0s. To do this, we find a threshold. Then when prob > threhsold we will predict 1 else 0. Let's search all thresholds and find which threshold is best.

(Note: this technique of finding the optimal threshold was first done by Mayukh Bhattacharyya here. My earlier notebooks used weird (and suboptimal) ways to convert probabilities into predictions. Using a threshold is the correct way to convert probabilties into predictions)

In [82]:
# map the computed question means to the question number
labels['m'] = labels.q_num.map(question_means)
labels['p1'] = labels.m.round()
labels.sample(5)


,session_id,correct,q_num,m,p1
176515,2090216134787132,1,15,0.482978,0.0
173948,1110323025157976,1,15,0.482978,0.0
166051,110315503653810,1,15,0.482978,0.0
208916,1100315382323988,1,18,0.950590,1.0
65344,1040609580658470,1,6,0.772052,1.0


In [83]:
m1 = f1_score(labels.correct, labels.p1, average='macro')
print('Rounding means achieves F1_SCORE =',m1)

Rounding means achieves F1_SCORE = 0.5857496919081725


In [84]:
labels['r'] = np.random.uniform(0,1,len(labels))
labels['p2'] = (labels.r < labels.m).astype('int32')
labels.sample(5)

,session_id,correct,q_num,m,p1,r,p2
9420,2000311233242510,1,1,0.724000,1.0,0.828165,0
63339,1020513480877012,1,6,0.772052,1.0,0.304060,1
197055,1100216031853000,1,17,0.685287,1.0,0.280792,1
8040,1090110403296550,0,1,0.724000,1.0,0.110877,1
194641,1040412021859204,1,17,0.685287,1.0,0.042061,1


In [85]:
m2 = f1_score( labels.correct, labels.p2, average='macro')
print('Random 1s and 0s achieves F1_SCORE =',m2)

Random 1s and 0s achieves F1_SCORE = 0.5711643499072043


In [86]:
use_random = []
for q_num in range(1,19):
    train['p3'] = labels.p1.copy()
    labels.loc[labels.q_num==q_num,'p3'] = labels.loc[labels.q_num==q_num,'p2']
    m = f1_score( labels.correct, labels.p3, average='macro')
    post = ''
    if m > m1:
        post = f': random better for q={q_num}'
        use_random.append(q_num)
    print(q_num,m,post)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
labels['p3'] = labels.p1.copy()
labels.loc[labels.q_num.isin(use_random),'p3'] = labels.loc[labels.q_num.isin(use_random),'p2']
m3 = f1_score(labels.correct, labels.p3, average='macro')
print('Using both Rounding and Random 1s 0s achieves F1_SCORE =',m3)